<a href="https://colab.research.google.com/github/nikitha-ramchander/Kaggle-Competition/blob/main/KaggleCompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My first try competing in a kaggle competition! 

In [369]:
# Imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#Train CSV
https://www.kaggle.com/c/csce5300-competition/data

In [370]:
# Load Train Data
bike_share = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Week 10: Gradient Boosting Machines /train_luc.csv')
bike_share.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [371]:
# Summary of DataFrame 
bike_share.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9174 entries, 0 to 9173
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    9174 non-null   object 
 1   season      9174 non-null   int64  
 2   holiday     9174 non-null   int64  
 3   workingday  9174 non-null   int64  
 4   weather     9174 non-null   int64  
 5   temp        9174 non-null   float64
 6   atemp       9174 non-null   float64
 7   humidity    9174 non-null   int64  
 8   windspeed   9174 non-null   float64
 9   casual      9174 non-null   int64  
 10  registered  9174 non-null   int64  
 11  count       9174 non-null   int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 860.2+ KB


In [372]:
# All datatypes are correct? 
bike_share.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [373]:
# Missing values? 
bike_share.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

In [374]:
# Duplicated rows? 
bike_share.duplicated().any()

False

In [375]:
# Before making changes lets duplcate the datetime column.
bike_share['date_time'] = bike_share['datetime'] 

In [376]:
# Check to see if it worked. 
bike_share.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9174 entries, 0 to 9173
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    9174 non-null   object 
 1   season      9174 non-null   int64  
 2   holiday     9174 non-null   int64  
 3   workingday  9174 non-null   int64  
 4   weather     9174 non-null   int64  
 5   temp        9174 non-null   float64
 6   atemp       9174 non-null   float64
 7   humidity    9174 non-null   int64  
 8   windspeed   9174 non-null   float64
 9   casual      9174 non-null   int64  
 10  registered  9174 non-null   int64  
 11  count       9174 non-null   int64  
 12  date_time   9174 non-null   object 
dtypes: float64(3), int64(8), object(2)
memory usage: 931.9+ KB


In [377]:
# Datetime column is important in this dataset, so lets figure out how to change it from an object datatype.
bike_share['datetime'] = pd.to_datetime(bike_share['datetime'])

In [378]:
# With the help of this article I understand how to split up the datetime column: 
# https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

bike_share['ScheduledDay_year'] = bike_share['datetime'].dt.year
bike_share['ScheduledDay_month'] = bike_share['datetime'].dt.month
bike_share['ScheduledDay_week'] = bike_share['datetime'].dt.isocalendar().week # got a future warning with dt.week so I have to include .isocalendar()
bike_share['ScheduledDay_day'] = bike_share['datetime'].dt.day
bike_share['ScheduledDay_hour'] = bike_share['datetime'].dt.hour
bike_share['ScheduledDay_minute'] = bike_share['datetime'].dt.minute
bike_share['ScheduledDay_dayofweek'] = bike_share['datetime'].dt.dayofweek

In [379]:
# I need to drop the datetime column here 
# But when I looked at the test.csv I noticed they didnt include columns casual and registered. 
# For me to run a model on .predict I need to drop these columns in this dataset
bike_share.drop(columns = ['datetime', 'casual', 'registered'], inplace= True)

# Check to see if it worked 
bike_share.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9174 entries, 0 to 9173
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season                  9174 non-null   int64  
 1   holiday                 9174 non-null   int64  
 2   workingday              9174 non-null   int64  
 3   weather                 9174 non-null   int64  
 4   temp                    9174 non-null   float64
 5   atemp                   9174 non-null   float64
 6   humidity                9174 non-null   int64  
 7   windspeed               9174 non-null   float64
 8   count                   9174 non-null   int64  
 9   date_time               9174 non-null   object 
 10  ScheduledDay_year       9174 non-null   int64  
 11  ScheduledDay_month      9174 non-null   int64  
 12  ScheduledDay_week       9174 non-null   UInt32 
 13  ScheduledDay_day        9174 non-null   int64  
 14  ScheduledDay_hour       9174 non-null   

In [380]:
# Next step is to set the date_time to an index so it doesnt affect the model. 
bike_share.set_index(['date_time'], inplace = True)

In [381]:
# Going to drop the Scheduled Day Week column since it wont work with the model 
bike_share.drop(columns = 'ScheduledDay_week', inplace = True )

In [382]:
# Check to see if all changes are in place. 
bike_share.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,ScheduledDay_year,ScheduledDay_month,ScheduledDay_day,ScheduledDay_hour,ScheduledDay_minute,ScheduledDay_dayofweek
date_time,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0,0,5
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1,0,5
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2,0,5
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3,0,5
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4,0,5


# Let's start modeling

In [383]:
# Split into features matrix and target vector 
X = bike_share.drop(columns= ['count'])
y = bike_share['count']

In [384]:
# 2-dimensional 
X.shape

(9174, 14)

In [385]:
# 1-dimensional 
y.shape

(9174,)

In [386]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = .3,
                                                    random_state = 42)

In [387]:
# Instantiate and fit gradient boosting classifier
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [388]:
# Calculate 
print('Training R^2:', gbr.score(X_train, y_train))
print('Testing R^2:', gbr.score(X_test, y_test))

Training R^2: 0.8760712909651382
Testing R^2: 0.868948184803543


In [389]:
# This competition needs RMSE: Root Mean Squared Error score 
print('Testing RMSE: ', np.sqrt(mean_squared_error(y_test, gbr.predict(X_test))))

Testing RMSE:  66.03712575442881


#Test CSV

In [390]:
# Load Test Data 
test = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Week 10: Gradient Boosting Machines /test_luc.csv')
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-17 00:00:00,1,1,0,2,8.20,9.850,47,15.0013
1,2011-01-17 01:00:00,1,1,0,2,8.20,9.850,44,12.9980
2,2011-01-17 02:00:00,1,1,0,2,7.38,8.335,43,16.9979
3,2011-01-17 03:00:00,1,1,0,2,7.38,9.090,43,12.9980
4,2011-01-17 04:00:00,1,1,0,2,7.38,9.850,43,8.9981


In [391]:
# Everything that I had done prior to fitting my model, I have to do the exact same steps to my Test.csv 
# Before making changes lets duplcate the datetime column.
test['date_time'] = test['datetime'] 

In [392]:
# Datetime column is important in this dataset, so lets figure out how to change it from an object datatype.
test['datetime'] = pd.to_datetime(test['datetime'])

In [393]:
# With the help of this article I understand how to split up the datetime column: 
# https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

test['ScheduledDay_year'] = test['datetime'].dt.year
test['ScheduledDay_month'] = test['datetime'].dt.month
test['ScheduledDay_week'] = test['datetime'].dt.isocalendar().week # got a future warning with dt.week so I have to include .isocalendar()
test['ScheduledDay_day'] = test['datetime'].dt.day
test['ScheduledDay_hour'] = test['datetime'].dt.hour
test['ScheduledDay_minute'] = test['datetime'].dt.minute
test['ScheduledDay_dayofweek'] = test['datetime'].dt.dayofweek

In [394]:
# I need to drop the datetime column here 
# This is when I noticed test.csv doesnt include the columns casual and registered. 
test.drop(columns = ['datetime'], inplace= True)

# Check to see if it worked 
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season                  1712 non-null   int64  
 1   holiday                 1712 non-null   int64  
 2   workingday              1712 non-null   int64  
 3   weather                 1712 non-null   int64  
 4   temp                    1712 non-null   float64
 5   atemp                   1712 non-null   float64
 6   humidity                1712 non-null   int64  
 7   windspeed               1712 non-null   float64
 8   date_time               1712 non-null   object 
 9   ScheduledDay_year       1712 non-null   int64  
 10  ScheduledDay_month      1712 non-null   int64  
 11  ScheduledDay_week       1712 non-null   UInt32 
 12  ScheduledDay_day        1712 non-null   int64  
 13  ScheduledDay_hour       1712 non-null   int64  
 14  ScheduledDay_minute     1712 non-null   

In [395]:
# Next step is to set the date_time to an index so it doesnt affect the model. 
test.set_index(['date_time'], inplace = True)

In [396]:
# Going to drop the Scheduled Day Week column since it wont work with the model 
test.drop(columns = 'ScheduledDay_week', inplace = True )

In [397]:
# Check to see if all changes are in place. 
test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,ScheduledDay_year,ScheduledDay_month,ScheduledDay_day,ScheduledDay_hour,ScheduledDay_minute,ScheduledDay_dayofweek
date_time,,,,,,,,,,,,,,
2011-01-17 00:00:00,1,1,0,2,8.20,9.850,47,15.0013,2011,1,17,0,0,0
2011-01-17 01:00:00,1,1,0,2,8.20,9.850,44,12.9980,2011,1,17,1,0,0
2011-01-17 02:00:00,1,1,0,2,7.38,8.335,43,16.9979,2011,1,17,2,0,0
2011-01-17 03:00:00,1,1,0,2,7.38,9.090,43,12.9980,2011,1,17,3,0,0
2011-01-17 04:00:00,1,1,0,2,7.38,9.850,43,8.9981,2011,1,17,4,0,0


In [398]:
# Let's generate new predictions 
preds = gbr.predict(test)
preds

array([  4.53542202,  -3.01044166, -17.13679217, ..., 209.20803071,
       170.27866659, 117.36667076])

In [399]:
# With my predictions I have to submit a file to the Kaggle Competition 
# This DataFrame has to be exact to what theyre asking for 
to_submit = pd.DataFrame({'datetime': test.index, 'count': preds})

In [400]:
# Check to see if it looks accurate 
to_submit.head()

,datetime,count
0,2011-01-17 00:00:00,4.535422
1,2011-01-17 01:00:00,-3.010442
2,2011-01-17 02:00:00,-17.136792
3,2011-01-17 03:00:00,-18.732908
4,2011-01-17 04:00:00,-19.780081


In [401]:
# Save this file and make sure to get rid of the index 
to_submit.to_csv('/content/drive/MyDrive/Coding Dojo/Week 10: Gradient Boosting Machines /submission.csv', index = False)